In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random as rd
import time
import datetime

In [4]:
import tensorflow as tf

print("Dostępne urządzenia GPU:", tf.config.list_physical_devices())


Dostępne urządzenia GPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
